# Transfer Learning Example on Radar Dataset (Great Smoky Mountains)

## Importing the required modules

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
import scipy.io as sio
import pandas as pd
import os
import re
import math
import itertools
import time
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
from itertools import product
from sklearn.utils.class_weight import compute_class_weight
torch.cuda.empty_cache()

# Reading the labels and creating train and test data

In [2]:
def index(string):
    s = re.findall("[0-9]", string)
    return int(''.join(s))

scenario_idx = 29 # Bonneville Salt Flats
names = os.listdir(f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names = sorted(names, key=index)
print(len(names))
y = pd.read_csv(f'num{scenario_idx}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names = y.columns[4:7]
y = y[col_names].to_numpy()

scenario_idx_test = 76 # Great Smoky Mountains
names_test = os.listdir(f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names_test = sorted(names_test, key=index)
y_test = pd.read_csv(f'num{scenario_idx_test}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names_test = y_test.columns[4:7]
y_test = y_test[col_names_test].to_numpy()

# Define fine-tuning name and labels
few_shot_count = 64
names_FS = names_test[:few_shot_count]
y_FS = y_test[:few_shot_count]

# Create training and testing datasets
y_train = y[:int(0.9 * len(names))]
y_test = y_test[int(0.9 * len(names_test)) - 1:]
training_names = names[:int(0.9 * len(names))]
test_names = names_test[int(0.9 * len(names_test)) - 1:]

print('Training labels:')
print(y_train)

# Tensor Corners
##################################################################################
# num29: [10851, 215, -5.4], num60: [11073, 215, -5.3], num62: [11471, 215, -5.6]
# num76: [11388, 215, -6.15], num35: [11381, 215, -0.95]
##################################################################################

# Training dataset global constants
coord_tr = [10851, 215, -5.4]   # Tensor corner
rng_res_tr = 59.9585 / 2        # Range resolution
az_step_tr = 0.4                # Azimuth step size
el_step_tr = 0.01               # Elevation step size

# Test dataset global constants
coord_ts = [11388, 215, -6.15]  # Tensor corner
rng_res_ts = 59.9585 / 2        # Range resolution
az_step_ts = 0.4                # Azimuth step size
el_step_ts = 0.01               # Elevation step size

def Drawing_Batch(names, label, bs, ind, directory, normalize=True):
    x = []
    labels = []
    
    for j in range(ind * bs, (ind + 1) * bs):
        try:
            temp = sio.loadmat(directory + names[j])['P']
        except:
            break
        if normalize:
            Anorm = temp - np.min(temp.flatten())
            temp = np.divide(Anorm, np.max(Anorm.flatten()))
        x.append(temp)
        labels.append(label[j, :])
        
    x = torch.FloatTensor(np.array(x))
    labels = torch.FloatTensor(np.array(labels))
    return x, labels

100000
Training labels:
[[ 8.3355 13.325  19.525 ]
 [18.156  17.388  32.775 ]
 [ 8.777  24.73   18.582 ]
 ...
 [15.785  15.992  29.641 ]
 [ 3.1736 10.716  12.27  ]
 [ 4.5552 21.035  14.234 ]]


In [3]:
print(len(y_train))
print(len(y_test))

90000
10001


## Define a regression CNN and instantiating it

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(21, 32, 3, 1)
        self.conv2 = nn.Conv1d(32, 64, 3, 1)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(64 * 5, 20)  # Adjust input size based on the output of conv layers and max pooling
        self.fc2_reg = nn.Linear(20, 2)  # Adjusted output size

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = F.max_pool1d(x, 2)
        
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = F.max_pool1d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        output_reg = self.fc2_reg(x)  # (bs, 2)
        
        return output_reg
    
from torchsummary import summary
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = Net()
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
print(summary(model,(21,26)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 24]           2,048
       BatchNorm1d-2               [-1, 32, 24]              64
            Conv1d-3               [-1, 64, 10]           6,208
       BatchNorm1d-4               [-1, 64, 10]             128
            Linear-5                   [-1, 20]           6,420
            Linear-6                    [-1, 2]              42
               Net-7                    [-1, 2]               0
Total params: 14,910
Trainable params: 14,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.06
Estimated Total Size (MB): 0.08
----------------------------------------------------------------
None


## Global Definitions

In [5]:
bs = 128  # batch_size
num_epoch = 10  # number of epochs
PATH = './ckpt_model.pth'   # forsaving the model
criterion = nn.MSELoss()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Define a Loss function and optimizer; Using GPU or CPU
model = Net()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    
def Spher2Cart_1D(spherical):
    cartesian = np.zeros(3)
    hypotenuse = np.cos(np.radians(spherical[2]))*spherical[0]
    cartesian[0] = np.cos(np.radians(spherical[1]))*hypotenuse
    cartesian[1] = -np.sin(np.radians(spherical[1]))*hypotenuse
    cartesian[2] = np.sin(np.radians(spherical[2]))*spherical[0]
    return cartesian

## Train and evaluate the network

In [6]:
def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

start = time.time()
main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True)
end = time.time()
print(end - start)

Epoch 0/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8207.26235776  7733.10778354 -1006.31216813]
[-8305.41473486  8082.7572811  -1144.22594828]
[-8801.11340659  7387.31225422 -1134.4771778 ]
Train Loss: 8.487845 ---- Test Loss: 64.618457
Epoch 1/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8205.59041975  7727.03001869 -1005.83167212]
[-8305.41473486  8082.7572811  -1144.22594828]
[-8789.46357174  7407.7753226  -1134.89738028]
Train Loss: 0.190858 ---- Test Loss: 63.072532
Epoch 2/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8205.22340794  7726.19132579 -1005.7565183 ]
[-8305.41473486  8082.7572811  -1144.22594828]
[-8784.78876847  7416.32199312 -1135.08864916]
Train Loss: 0.134331 ---- Test Loss: 58.491567
Epoch 3/10
----------
[-8187.45955885  7784.83170223 -1008.20172776]
[-8208.47782392  7721.71008594 -1005.69392866]
[-8305.41473486  8082.7572811  -1144.22594828]
[-8797.67291338  7400.38480342 -1135.04741968]
Train Loss: 0.11516

In [7]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 8.37928448728461
Localization Error (m) = 691.347446648395


## Fine-Tuning Regression CNN

In [8]:
model.module.conv1.weight.requires_grad = False
model.module.conv1.bias.requires_grad = False
model.module.batchnorm1.weight.requires_grad = False
model.module.batchnorm1.bias.requires_grad = False

model.module.conv2.weight.requires_grad = False
model.module.conv2.bias.requires_grad = False
model.module.batchnorm2.weight.requires_grad = False
model.module.batchnorm2.bias.requires_grad = False

In [9]:
bs = 64  # batch_size
num_epoch = 100
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20  # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names) // bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:, 0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)

        # Training error display
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1], [rng_res_tr, az_step_tr, el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names) // bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:, 0:2])
                running_loss_test += loss_test.item()
                
        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)

        # Test error display
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1], [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)

        epoch_loss_train = running_loss_train * x_train.size(0) / len(training_names)
        epoch_loss_test = running_loss_test * x_test.size(0) / len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch % 5 == 0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_error = epoch_loss_test

main(names_FS, test_names, bs, num_epoch, y_FS, y_test, normalize=True)

Epoch 0/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-7920.53664558  7806.68647488  -951.78932896]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8712.19533732  7481.03148732 -1127.96127404]
Train Loss: 24.690163 ---- Test Loss: 47.718513
Epoch 1/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-7904.51969873  7813.41984623  -951.21855363]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8684.00799282  7487.68242512 -1126.28865401]
Train Loss: 23.312172 ---- Test Loss: 46.488462
Epoch 2/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-7889.87712386  7820.05226781  -950.72721716]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8652.16414354  7509.63447894 -1125.33382938]
Train Loss: 22.059099 ---- Test Loss: 44.359572
Epoch 3/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-7877.16607978  7826.7241956   -950.35725994]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8620.40347791  7536.68340015 -1124.72664265]
Train Loss: 

[-8271.55879786  8100.10882815 -1137.17137312]
[-8326.44976597  7956.15273643 -1131.21533739]
Train Loss: 8.640176 ---- Test Loss: 12.882621
Epoch 33/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8029.57402433  7734.83676055  -954.1816026 ]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8326.13618562  7957.80610266 -1131.30527177]
Train Loss: 8.592516 ---- Test Loss: 12.753633
Epoch 34/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8032.93037226  7733.36098559  -954.30089983]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8325.54503839  7959.5670243  -1131.38281655]
Train Loss: 8.553005 ---- Test Loss: 12.636240
Epoch 35/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8035.62960983  7732.45461278  -954.41354872]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8324.80664974  7961.39162333 -1131.45425595]
Train Loss: 8.520977 ---- Test Loss: 12.529365
Epoch 36/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8037.8326

[-8271.55879786  8100.10882815 -1137.17137312]
[-8337.82808512  7964.57202865 -1132.59472913]
Train Loss: 8.249731 ---- Test Loss: 12.045780
Epoch 65/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8054.56830392  7751.92445949  -956.7368741 ]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8337.15908676  7964.76996615 -1132.56064303]
Train Loss: 8.242723 ---- Test Loss: 12.046656
Epoch 66/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8052.86256335  7753.43564655  -956.72139542]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8336.62528034  7964.8744869  -1132.52982271]
Train Loss: 8.236296 ---- Test Loss: 12.049071
Epoch 67/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8051.317809    7754.81886063  -956.70828158]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8336.2454662   7964.89296032 -1132.50410162]
Train Loss: 8.230391 ---- Test Loss: 12.052647
Epoch 68/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8049.9586

[-8271.55879786  8100.10882815 -1137.17137312]
[-8332.35914984  7966.22791808 -1132.31873815]
Train Loss: 8.146729 ---- Test Loss: 12.048567
Epoch 97/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8033.01189834  7776.72815956  -956.88376916]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8332.36227942  7966.31479584 -1132.3248575 ]
Train Loss: 8.144920 ---- Test Loss: 12.043524
Epoch 98/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8032.9461083   7777.00617159  -956.89627352]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8332.34378055  7966.43493855 -1132.33169931]
Train Loss: 8.143171 ---- Test Loss: 12.037817
Epoch 99/100
----------
[-8039.80337222  7765.79211219  -956.65097951]
[-8032.86596034  7777.30179043  -956.90894365]
[-8271.55879786  8100.10882815 -1137.17137312]
[-8332.29236592  7966.58328973 -1132.3381191 ]
Train Loss: 8.141479 ---- Test Loss: 12.031713


In [10]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical), 3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:, 2])), spherical[:, 0])
    cartesian[:, 0] = np.multiply(np.cos(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 1] = np.multiply(-np.sin(np.radians(spherical[:, 1])), hypotenuse)
    cartesian[:, 2] = np.multiply(np.sin(np.radians(spherical[:, 2])), spherical[:, 0])
    return cartesian

# Testing: (range, az, el)
model.eval()
out_test_reg = np.zeros((len(y_test), 3))
labels_test_reg = np.zeros((len(y_test), 3))

for i in range(0, len(y_test) // bs):
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, f'num{scenario_idx_test}_NAMF_DATA_20_rng30_pulse1_1000_100k/', True)
    x_test = x_test.to(device)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs * i: bs * i + bs] = (labels_test[:, 0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs * i: bs * i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs * i: bs * i + bs, 2] = labels_test_reg[bs * i: bs * i + bs, 2]

# Calculate azimuth estimation error
azim_tot = 0
for i in range(len(out_test_reg)):
    azim_tot += np.linalg.norm(out_test_reg[i, 1] - labels_test_reg[i, 1])

azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

# Convert spherical coordinates to Cartesian coordinates
new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts, az_step_ts, el_step_ts]) + coord_ts)

# Calculate localization error
sum_tot = 0
for i in range(0, len(new_data) - (len(new_data) % bs), 1):
    sum_tot += np.linalg.norm(new_data[i, :] - new_labels_data[i, :])

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 1.782160221439709
Localization Error (m) = 176.62557058374003
